In [ ]:
%pip install openai

In [9]:
import openai
import os
import pandas as pd
import json
import re

In [5]:
filepath = "../data/retractable.json"
with open(filepath, 'r') as fp:
    data = json.load(fp)

df = pd.DataFrame.from_dict(data["retracted_articles"])
retrieved = df[df['retraction_body'] != ""]


In [6]:
retrieved

,article,pmcid,url,retraction_reason
1,MED36961293,PMC10054288,https://europepmc.org/article/MED/36961293,"{""concerned reader"":""We, the Editors and Publi..."
11,MED37106383,PMC10142152,https://europepmc.org/article/MED/37106383,"{""concerned reader"":""Retraction Note: Journal ..."
21,PMC10139839,PMC10139839,https://europepmc.org/article/PMC/PMC10139839,"{""research misconduct and data manipulation"":""..."
26,MED37093887,PMC10124820,https://europepmc.org/article/MED/37093887,"{""concerned reader"":""Following the publication..."
28,MED37106357,PMC10142444,https://europepmc.org/article/MED/37106357,"{""concerned reader"":""Retraction Note: Mol Canc..."
...,...,...,...,...
14516,MED195582,PMC1164788,https://europepmc.org/article/MED/195582,"{""unavailable"":""""}"
14518,MED34959264,PMC323952,https://europepmc.org/article/MED/34959264,"{""unavailable"":""""}"
14521,MED16578848,PMC400275,https://europepmc.org/article/MED/16578848,"{""unavailable"":""""}"
14524,MED9066475,PMC2126087,https://europepmc.org/article/MED/9066475,"{""unavailable"":""""}"


In [7]:
with open('assets/openAI_key.txt', 'r') as f:
    api_key = f.read().strip()

os.environ['OPENAI_API_KEY'] = api_key

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
# Define the prompt to identify retraction notices
notice_prompt = "Please identify if the following text is a retraction notice:"

# Define the prompt to extract retraction reasons
reason_prompt = "Please extract the reason for retraction from the following text:"

# Define the list of retraction notices
notices = [
    "invalid results",
    "plagiarism",
    "conflict of interest, legal reasons",
    "research misconduct and data manipulation",
    "agreement by author(s)",
    "reader concerns",
    "issues with authorship",
    "duplicated paper",
    "could not reproduce results",
]

# Define a dictionary to store the results
results = {
    "invalid results": [],
    "plagiarism": [],
    "conflict of interest, legal reasons": [],
    "research misconduct and data manipulation": [],
    "agreement by author(s)": [],
    "reader concerns": [],
    "issues with authorship": [],
    "duplicated paper": [],
    "could not reproduce results": []
}

In [ ]:

# Loop over the retraction notices
for entry in data:
    notice = entry["retraction_body"]
    # Generate text to identify the retraction notice
    identification_text = notice_prompt + " " + notice
    identification_result = openai.Completion.create(
        engine="davinci",
        prompt=identification_text,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.7,
    )
    is_notice = identification_result.choices[0].text.strip().lower() == "yes"

    # If the text is a retraction notice, extract the reasons
    if is_notice:
        reason_text = reason_prompt + " " + notice
        reason_result = openai.Completion.create(
            engine="davinci",
            prompt=reason_text,
            max_tokens=1024,
            n=1,
            stop=None,
            temperature=0.7,
        )
        reason_string = reason_result.choices[0].text.strip().lower()
        for reason in results.keys():
            if reason in reason_string:
                match = re.search(reason + r".*", reason_string)
                results[reason].append(match.group())

# Print the results
print(results)

# Write the results to a file
with open("../data/davinci_retract_reason.json", "w") as f:
    json.dump(results, f)

# Print a success message
print("Results written to file.")